# Tuning

Importazione librerie

In [1]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_tuner import HyperModel, Hyperband
from sklearn.model_selection import KFold
from sklearn.utils import check_random_state

import utility as ut
from tfkan import DenseKAN

Impostiamo il random state

In [2]:
# Valore del seme causale
seed_value = 0

# Impostazione dei semi casuali per os, random, numpy e tensorflow
os.environ['PYTHONHASHSEED'] = str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

# Controllo che il seme sia stato correttamente impostato
check_random_state(seed_value)

RandomState(MT19937) at 0x1EC5143BD40

Prepariamo il dataset 

In [3]:
x_train = pd.read_csv("datasets/x_train.csv")   # Caricamento del dataset
y_train = pd.read_csv("datasets/y_train.csv")
x_test = pd.read_csv("datasets/x_test.csv")  
y_test = pd.read_csv("datasets/y_test.csv")

ut.standardize(x_train)                         # Standardizzazione
x_train = x_train.to_numpy()[:, 1:-1]
y_train = y_train.to_numpy()[:, -1]
ut.standardize(x_test)                   
x_test = x_test.to_numpy()[:, 1:-1]
y_test = y_test.to_numpy()[:, -1]

x_train = tf.convert_to_tensor(x_train, dtype=tf.float32)   # Conversione a tensore
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test = tf.convert_to_tensor(x_test, dtype=tf.float32)  
y_test = tf.convert_to_tensor(y_test, dtype=tf.float32)


## Tuner architettura KAN

In [4]:
class HyperKAN(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))

        num_layers = hp.Int('num_layers', 1, 3)
        for i in range(num_layers):
            grid_range = hp.Float(f'grid_range_min_{i}', 1.0, 4.0)
            model.add(DenseKAN(
                units=hp.Int(f'units_{i}', 1, 16),
                grid_size=hp.Int(f'grid_size_{i}', 8, 32),
                grid_range=[
                    -grid_range,
                    grid_range
                ]
            ))
        
        model.add(DenseKAN(1))

        learning_rate = hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')
        optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])
        
        opt = tf.keras.optimizers.get(optimizer)
        opt.learning_rate = learning_rate

        model.compile(
            optimizer=opt,
            loss='mean_absolute_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
        )
        return model


def run_tuner(x_train, y_train, x_test, y_test, input_shape):
    hypermodel = HyperKAN(input_shape=input_shape)

    tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=50,
        directory="./modelli_salvati",
        project_name='retina_kan'
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    kfold = KFold(n_splits=5, shuffle=True, random_state=0)

    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train.numpy())):
        print(f"Fold {fold + 1}")
        
        x_train_fold = tf.gather(x_train, train_indices)
        x_val_fold = tf.gather(x_train, val_indices)
        y_train_fold = tf.gather(y_train, train_indices)
        y_val_fold = tf.gather(y_train, val_indices)

        tuner.search(
            x_train_fold, y_train_fold,
            epochs=100,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[early_stopping]
        )

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_retina_kan_model.h5')

    return best_model

N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_test, y_test, input_shape=(N_FEATURES,))

Trial 6 Complete [00h 00m 16s]
val_loss: 3.4124507904052734

Best val_loss So Far: 0.4860513210296631
Total elapsed time: 00h 01m 44s

Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_layers
2.5974            |2.4101            |grid_range_min_0
9                 |14                |units_0
21                |23                |grid_size_0
0.0077505         |0.0016175         |learning_rate
rmsprop           |rmsprop           |optimizer
2.2659            |1                 |grid_range_min_1
1                 |1                 |units_1
29                |8                 |grid_size_1
2.8216            |1                 |grid_range_min_2
12                |1                 |units_2
29                |8                 |grid_size_2
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0   

KeyboardInterrupt: 

## Tuner architettura MLP

In [5]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Input(shape=self.input_shape))

        num_layers = hp.Int('num_layers', 1, 5)
        for i in range(num_layers):
            model.add(tf.keras.layers.Dense(
                units=hp.Int(f'units_{i}', 4, 256, step=4),
                activation=hp.Choice(f'activation_{i}', ['relu', 'elu', 'selu']),
                kernel_regularizer=tf.keras.regularizers.l2(hp.Float(f'l2_{i}', 1e-4, 1e-2, sampling='log'))
            ))

            use_dropout = hp.Boolean(f'use_dropout_{i}')
            if use_dropout:
                model.add(tf.keras.layers.Dropout(rate=0.5))

        
        model.add(tf.keras.layers.Dense(1))

        learning_rate = hp.Float('learning_rate', 1e-4, 1e-1, sampling='log')
        optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'adadelta', 'adagrad'])

        opt = tf.keras.optimizers.get(optimizer)
        opt.learning_rate = learning_rate

        model.compile(
            optimizer=opt,
            loss='mean_absolute_error',
            metrics=[tf.keras.metrics.MeanAbsoluteError(name='mae')]
        )
        return model


def run_tuner(x_train, y_train, x_test, y_test, input_shape):
    hypermodel = MyHyperModel(input_shape=input_shape)

    tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=50,
        directory="./modelli_salvati",
        project_name='retina_mlp',

    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    kfold = KFold(n_splits=5, shuffle=True, random_state=0)

    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)

    for fold, (train_indices, val_indices) in enumerate(kfold.split(x_train.numpy())):
        print(f"Fold {fold + 1}")
        
        x_train_fold = tf.gather(x_train, train_indices)
        x_val_fold = tf.gather(x_train, val_indices)
        y_train_fold = tf.gather(y_train, train_indices)
        y_val_fold = tf.gather(y_train, val_indices)

        tuner.search(
            x_train_fold, y_train_fold,
            epochs=50,
            validation_data=(x_val_fold, y_val_fold),
            callbacks=[early_stopping]
        )

    tuner.results_summary()
    best_model = tuner.get_best_models(num_models=1)[0]

    test_loss, test_mae = best_model.evaluate(x_test, y_test)
    print(f"Test MAE: {test_mae}")

    best_model.save('best_retina_kan_model.h5')

    return best_model

# Assicurati che x_train, y_train, x_test, y_test siano definiti correttamente
N_FEATURES = x_train.shape[1]
best_model = run_tuner(x_train, y_train, x_test, y_test, input_shape=(N_FEATURES,))


Trial 16 Complete [00h 00m 08s]
val_loss: 0.775733232498169

Best val_loss So Far: 0.769001305103302
Total elapsed time: 22h 57m 16s

Search: Running Trial #17

Value             |Best Value So Far |Hyperparameter
4                 |3                 |num_layers
112               |112               |units_0
elu               |relu              |activation_0
0.00011383        |0.0010242         |l2_0
True              |True              |use_dropout_0
0.013722          |0.0006935         |learning_rate
adam              |adam              |optimizer
196               |108               |units_1
relu              |relu              |activation_1
0.00021434        |0.0015578         |l2_1
False             |True              |use_dropout_1
152               |28                |units_2
selu              |selu              |activation_2
0.00044919        |0.00018398        |l2_2
True              |False             |use_dropout_2
172               |8                 |units_3
elu            

KeyboardInterrupt: 

# Tuning modelli Machine Learning
Tutti i punteggi seguono la convenzione secondo cui valori restituiti più alti sono migliori di valori restituiti più bassi. Pertanto le metriche che misurano la distanza tra il modello e i dati, come metrics.mean_squared_error, sono disponibili come neg_mean_squared_error che restituiscono il valore negato della metrica.